In [1]:
import torch

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

checkpoint = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
    pad_token_id=tokenizer.eos_token_id)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

ecomm = {
    "type": "object",
    "properties": {
        "store": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "location": {"type": "string"},
                "inventory": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "productId": {"type": "string"},
                            "name": {"type": "string"},
                            "description": {"type": "string"},
                            "category": {"type": "string"},
                            "price": {"type": "number"},
                            "inStock": {"type": "boolean"},
                            "rating": {"type": "number"},
                            "images": {"type": "array", "items": {"type": "string"}},
                        },
                    },
                },
            },
        }
    },
}


builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=ecomm,
    prompt="write a description about mike's ski shop which sells premium skis and snowboards",
    max_string_token_length=20,
    debug=True
)

print("Generating...")
output = builder()

highlight_values(output)

Generating...
[generate_object] generating value for store
[generate_object] generating value for name
[generate_string] write a description about mike's ski shop which sells premium skis and snowboards
Output result in the following JSON schema format:
{"type": "object", "properties": {"store": {"type": "object", "properties": {"name": {"type": "string"}, "location": {"type": "string"}, "inventory": {"type": "array", "items": {"type": "object", "properties": {"productId": {"type": "string"}, "name": {"type": "string"}, "description": {"type": "string"}, "category": {"type": "string"}, "price": {"type": "number"}, "inStock": {"type": "boolean"}, "rating": {"type": "number"}, "images": {"type": "array", "items": {"type": "string"}}}}}}}}}
Result: {"store": {"name": "
[generate_string] |Mike's Ski Shop",|
[generate_object] generating value for location
[generate_string] write a description about mike's ski shop which sells premium skis and snowboards
Output result in the following JSON s

In [3]:
car = {
    "type": "object",
    "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "number"},
        "colors_available": {
            "type": "array",
            "items": {"type": "string"},
        },
    },
}

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="generate an example car",
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  make: "Ford",
  model: "F-150",
  year: 2017.0,
  colors_available: [
    "Blue",
    "White",
    "Red",
    "Black"
  ]
}


In [4]:
complex_car = {"type": "object", "properties": {"car": {"type": "object", "properties": {"make": {"type": "string"}, "model": {"type": "string"}, "year": {"type": "number"}, "colors": {"type": "array", "items": {"type": "string"}}, "features": {"type": "object", "properties": {"audio": {"type": "object", "properties": {"brand": {"type": "string"}, "speakers": {"type": "number"}, "hasBluetooth": {"type": "boolean"}}}, "safety": {"type": "object", "properties": {"airbags": {"type": "number"}, "parkingSensors": {"type": "boolean"}, "laneAssist": {"type": "boolean"}}}, "performance": {"type": "object", "properties": {"engine": {"type": "string"}, "horsepower": {"type": "number"}, "topSpeed": {"type": "number"}}}}}}}, "owner": {"type": "object", "properties": {"firstName": {"type": "string"}, "lastName": {"type": "string"}, "age": {"type": "number"}}}}}
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=complex_car,
    prompt="generate an example Rolls Royce Phantom",
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  car: {
    make: "Rolls Royce",
    model: "Phantom",
    year: 2015.0,
    colors: [
      "Silver",
      "Black",
      "Red",
      "Blue",
      "Grey",
      "White",
      "Green",
      "Yellow",
      "Brown",
      "Purple"
    ],
    features: {
      audio: {
        brand: "Bose",
        speakers: 12.0,
        hasBluetooth: True
      },
      safety: {
        airbags: 6.0,
        parkingSensors: False,
        laneAssist: False
      },
      performance: {
        engine: "V12",
        horsepower: 563.0,
        topSpeed: 155.0
      }
    }
  },
  owner: {
    firstName: "John",
    lastName: "Doe",
    age: 32.0
  }
}
